In [1]:
import requests
import pandas as pd
import numpy as np
import time
import pickle
import json
import os
import random

In [2]:
def to_timestamp(date):
    return int(date.timestamp())

In [3]:
def run_query(api, query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post(api, json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))

In [4]:
def get_market_query(block):
    query = '''{
      markets(block: {number: ''' + str(block) + '''}, first: 1000) {
        id
        protocol { id }
        name

        isActive
        canBorrowFrom
        canUseAsCollateral
        maximumLTV
        liquidationThreshold
        liquidationPenalty

        createdTimestamp
        createdBlockNumber

        rewardTokens { id }
        rewardTokenEmissionsAmount
        rewardTokenEmissionsUSD
        _stakedOutputTokenAmount

        inputToken  { id symbol }
        inputTokenBalance
        inputTokenPriceUSD

        outputToken { id symbol }
        outputTokenSupply
        outputTokenPriceUSD

        exchangeRate
        rates {
          id
          rate
          duration
          maturityBlock
          side
          type

        }

        _reserveFactor
        totalValueLockedUSD
        totalDepositBalanceUSD
        totalBorrowBalanceUSD
        
        _borrowBalance
        _rewardLastUpdatedTimestamp
        _cumulativeRewardAmount
        _stakedOutputTokenAmount
      }
    }'''
    
    return run_query(api, query)['data']['markets']

In [5]:
d = dict()
d['AAVE V2 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/aave-v2-ethereum', 11363052)
d['AAVE V3 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/aave-v3-ethereum', 16496817)
d['Compound V2 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/compound-v2-ethereum', 7710833)
d['Compound V3 Ethereum'] = ('https://api.thegraph.com/subgraphs/name/messari/compound-v3-ethereum', 15412361)

In [6]:
subgraph = 'Compound V2 Ethereum'
api, start_block = d[subgraph]
folder = f'Messari/{subgraph}/markets'

In [7]:
blocks = pd.read_csv('data/blocks_hourly.csv')
blocks['time'] = pd.to_datetime(blocks['time'])

blocks = blocks[blocks['max_block'] > start_block]

In [8]:
for name, group in blocks.groupby(pd.Grouper(key='time', freq='1M')):
    if os.path.isfile(f'{folder}/markets_{name.strftime("%Y-%m-%d")}.json'):
        continue
    
    temp = dict()
    for idx, row in group.iterrows():
        if '00:00:00' in str(row['time']):
            print(str(row['time']))
            time.sleep(10)
            
        while True:
            try:
                temp[str(row['time'])] = get_market_query(row['max_block'])
                break
            except Exception as e:
                print(e)
                time.sleep(60)
                continue
    
    with open(f'{folder}/markets_{name.strftime("%Y-%m-%d")}.json', 'w') as outfile:
        outfile.write(json.dumps(temp))
    
    time.sleep(30)

2023-12-01 00:00:00
2023-12-02 00:00:00
2023-12-03 00:00:00
2023-12-04 00:00:00
2023-12-05 00:00:00
2023-12-06 00:00:00
2023-12-07 00:00:00
2023-12-08 00:00:00
2023-12-09 00:00:00
2023-12-10 00:00:00
('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))
2023-12-11 00:00:00
2023-12-12 00:00:00
2023-12-13 00:00:00
2023-12-14 00:00:00
2023-12-15 00:00:00
2023-12-16 00:00:00
2023-12-17 00:00:00
2023-12-18 00:00:00
2023-12-19 00:00:00
2023-12-20 00:00:00
2023-12-21 00:00:00
2023-12-22 00:00:00
2023-12-23 00:00:00
2023-12-24 00:00:00
2023-12-25 00:00:00
2023-12-26 00:00:00
2023-12-27 00:00:00
2023-12-28 00:00:00
2023-12-29 00:00:00
2023-12-30 00:00:00
2023-12-31 00:00:00
